<a href="https://colab.research.google.com/github/b-hahn/deepmind-research/blob/master/rl_unplugged/atari_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RL Unplugged: Offline DQN - Atari
## Guide to  training an Acme DQN agent on Atari data.
# <a href="https://colab.research.google.com/github/deepmind/deepmind_research/blob/master/rl_unplugged/atari_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




## Installation

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install numpy>=1.20.0
!pip install dm-acme
!pip install dm-acme[reverb]
!pip install dm-reverb
!pip install dm-acme[tf]
!pip install dm-sonnet
!pip install dopamine-rl==3.1.2
!pip install atari-py

!git clone https://github.com/deepmind/deepmind-research.git
%cd deepmind-research

     |████████████████████████████████| 309 kB 5.4 MB/s 
     |████████████████████████████████| 3.8 MB 56.4 MB/s 
  Created wheel for dm-acme: filename=dm_acme-0.3.0-py3-none-any.whl size=561406 sha256=e2e827c1a0ebd0e49e91e6a72e8c007f8d90361d7dbc271b92a9f3de17d81cf7
  Stored in directory: /root/.cache/pip/wheels/a8/ae/a1/63409d662e89b8c25d080c1e897253b51d0630e2fba790cf6b
Successfully built dm-acme
     |████████████████████████████████| 6.4 MB 4.7 MB/s 
     |████████████████████████████████| 254 kB 5.3 MB/s 
     |████████████████████████████████| 104 kB 69.2 MB/s 
     |████████████████████████████████| 4.0 MB 63.3 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
     |████████████████████████████████| 111 kB 5.4 MB/s 
     |████████████████████████████████| 21.8 MB 3.6 MB/s 
     |████████████████████████████████| 352 kB 58

## Imports

In [11]:
import copy

import acme
from acme.agents.tf import actors
from acme.agents.tf.dqn import learning as dqn
from acme.tf import utils as acme_utils
from acme.utils import loggers

# import gin
# gin.enter_interactive_mode()
!pip install gin-config==0.3.0
from rl_unplugged import atari
import sonnet as snt
import tensorflow as tf

## Data

In [12]:
game = 'Pong' #@param
run = 1  #@param

tmp_path = '/tmp/atari'
gs_path = 'gs://rl_unplugged/atari'

!mkdir -p {tmp_path}/{game}

src = f'{gs_path}/{game}/run_{run}-00000-of-00100'
dest = f'{tmp_path}/{game}/run_{run}-00000-of-00001'
!gsutil cp {src} {dest}

Copying gs://rl_unplugged/atari/Pong/run_1-00000-of-00100...
- [1 files][720.0 MiB/720.0 MiB]   56.7 MiB/s                                   
Operation completed over 1 objects/720.0 MiB.                                    


## Dataset and environment

In [56]:
batch_size = 10  

# def discard_extras(sample):
#   return sample._replace(data=sample.data[:5])
from acme import types
import reverb
# from collections import namedtuple
from typing import NamedTuple, List, Any


dataset = atari.dataset(path=tmp_path, game='Pong', run=1, num_shards=1)
# # Small batch size, experiments in the paper were run with batch size 256.
# dataset = dataset.map(discard_extras).batch(batch_size)
class testClass(NamedTuple):
  some_field: int
  other_field: List[int]

NestedArray = Any
NestedTensor = Any

class copyTransition(NamedTuple):
  observation: NestedArray
  action: NestedArray
  reward: NestedArray
  discount: NestedArray
  next_observation: NestedArray
  next_action: NestedArray
  extras: NestedArray = ()

def convert_to_transition(sample):
  # transition_sample = types.Transition
  sample = sample._replace(data=copyTransition(observation = sample.data[0],
                                     action = sample.data[1],
                                     reward = sample.data[2],
                                     discount = sample.data[3],
                                     next_observation = sample.data[4],
                                     next_action = sample.data[5],
                                     extras = sample.data[6]))
  # transition_sample = namedtuple('Transition', ['observation', 'action', 'reward', 'discount', 'next_observation', 'next_action', 'extra'])
  # transition_sample.observation = sample.data[0]
  # transition_sample.action = sample.data[1]
  # transition_sample.reward = sample.data[2]
  # transition_sample.discount = sample.data[3]
  # transition_sample.next_observation = sample.data[4]
  # transition_sample.next_action = sample.data[5]
  # transition_sample.extra = sample.data[6]
  # transition_sample = tf.nest.map_structure(tf.convert_to_tensor, transition_sample)
  # transition_sample = reverb.ReplaySample(info=None, data=sample.data)
  # transition_sample = testClass(some_field = 7, other_field = [1, 4, 2])
  # return transition_sample
  return sample

dataset = dataset.map(convert_to_transition).batch(batch_size)

# print(len(next(iter(dataset)).data))
print(dir(next(iter(dataset))))
print(dir(next(iter(dataset)).data))


['__add__', '__annotations__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_field_defaults', '_field_types', '_fields', '_fields_defaults', '_make', '_replace', 'count', 'data', 'index', 'info']
['__add__', '__annotations__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', 

In [16]:
!python -m atari_py.import_roms '/content/drive/MyDrive/Colab Notebooks/atari_roms'
environment = atari.environment(game='Pong')

copying breakout.bin from /content/drive/MyDrive/Colab Notebooks/atari_roms/Breakout - Breakaway IV (Paddle) (1978) (Atari, Brad Stewart - Sears) (CX2622 - 6-99813, 49-75107) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/breakout.bin
copying pong.bin from /content/drive/MyDrive/Colab Notebooks/atari_roms/Video Olympics - Pong Sports (Paddle) (1977) (Atari, Joe Decuir - Sears) (CX2621 - 99806, 6-99806, 49-75104) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/pong.bin


## DQN learner

In [17]:
# Get total number of actions.
num_actions = environment.action_spec().num_values

# Create the Q network.
network = snt.Sequential([
    lambda x: tf.image.convert_image_dtype(x, tf.float32),
    snt.Conv2D(32, [8, 8], [4, 4]),
    tf.nn.relu,
    snt.Conv2D(64, [4, 4], [2, 2]),
    tf.nn.relu,
    snt.Conv2D(64, [3, 3], [1, 1]),
    tf.nn.relu,
    snt.Flatten(),
    snt.nets.MLP([512, num_actions])
])
acme_utils.create_variables(network, [environment.observation_spec()])

TensorSpec(shape=(6,), dtype=tf.float32, name=None)

In [57]:
# Create a logger.
logger = loggers.TerminalLogger(label='learner', time_delta=1.)

# Create the DQN learner.
learner = dqn.DQNLearner(
    network=network,
    target_network=copy.deepcopy(network),
    discount=0.99,
    learning_rate=3e-4,
    importance_sampling_exponent=0.2,
    target_update_period=2500,
    dataset=dataset,
    logger=logger)

## Training loop

In [58]:
for _ in range(100):
  learner.step()

INFO:tensorflow:Assets written to: /root/acme/bd9054ec-7850-11ec-96c8-0242ac1c0002/snapshots/network/assets


## Evaluation

In [59]:
# Create a logger.
logger = loggers.TerminalLogger(label='evaluation', time_delta=1.)

# Create an environment loop.
policy_network = snt.Sequential([
    network,
    lambda q: tf.argmax(q, axis=-1),
])
loop = acme.EnvironmentLoop(
    environment=environment,
    actor=actors.FeedForwardActor(policy_network=policy_network),
    logger=logger)

loop.run(5)